In [1]:
import opendatasets as od
import pandas as pd
import os

import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers,regularizers,losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

2023-07-27 19:58:28.925129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# dataset = "https://www.kaggle.com/datasets/gpiosenka/us-license-plates-image-classification"

In [ ]:
# od.download(dataset, force=True)

In [2]:
train_data = ImageDataGenerator(rescale=1/255.0,
                                preprocessing_function=preprocess_input)

test_data = ImageDataGenerator(rescale=1/255.0,
                               preprocessing_function=preprocess_input)

valid_data = ImageDataGenerator(rescale=1/255.0,
                                preprocessing_function=preprocess_input)


In [ ]:
trainImageData = train_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/train",
                                                batch_size=4,
                                                class_mode="categorical",
                                                target_size=(128,256))

testImageData = valid_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/test",
                                                batch_size=4,
                                                class_mode="categorical",
                                                target_size=(128,256))

validImageData = valid_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/valid",
                                                batch_size=4,
                                                class_mode="categorical",
                                                target_size=(128,256))

In [ ]:
trainImageData.image_shape

In [ ]:
classes = len(trainImageData.class_indices.values())


In [ ]:
for _ in range(1):
    img, label = trainImageData.next()
    print(img.shape)
    print(label[0])
    plt.imshow(img[0])
    plt.show()

In [ ]:

def build_model():
    model = models.Sequential()
    NO_CLASSES = classes

    model.add(layers.Conv2D(32, (3,3), input_shape=trainImageData.image_shape,
              padding='same', activation='relu'))
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    # model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(NO_CLASSES, activation='softmax'))

    return model

In [ ]:
model = build_model()

model.compile(loss='categorical_crossentropy',
              optimizer='ADAM', metrics=['accuracy'])

model.summary()
hist=model.fit(trainImageData, validation_data=validImageData, epochs=10)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
vgg16_model = VGG16(weights="imagenet",include_top=False, input_shape= trainImageData.image_shape)
vgg16_model.trainable = False
# train_ds = preprocess_input(trainImageData)
# valid_ds = preprocess_input(validImageData)

In [ ]:
flatten_layer = layers.Flatten()
dense_1 = layers.Dense(64, activation='relu')
dense_2 = layers.Dense(64, activation='relu')
dense_3 = layers.Dense(classes, activation='softmax')


vgg16_sequential_model = models.Sequential([
    vgg16_model,
    flatten_layer,
    dense_1,
    dense_2,
    dense_3
])



In [ ]:
vgg16_sequential_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_sequential_model.fit(trainImageData,validation_data=validImageData, epochs=25)

In [8]:
trainImageResnet = train_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/train",
                                                batch_size=32,
                                                class_mode="categorical",
                                                target_size=(128,256))

testImageResnet = valid_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/test",
                                                batch_size=32,
                                                class_mode="categorical",
                                                target_size=(128,256))

validImageResnet = valid_data.flow_from_directory("../../Datasets/us-license-plates-image-classification/new plates/valid",
                                                batch_size=32,
                                                class_mode="categorical",
                                                target_size=(128,256))

Found 8161 images belonging to 56 classes.
Found 280 images belonging to 56 classes.
Found 280 images belonging to 56 classes.


In [4]:
resNet50_model = ResNet50(weights="imagenet", include_top=False, input_shape= trainImageResnet.image_shape)
resNet50_model.trainable = False

2023-07-27 19:58:40.260947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 19:58:40.262508: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
flatten_layer_resnet = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dense_layer_2 = layers.Dense(56, activation='softmax')

resnet50_sequential_model = models.Sequential([
    resNet50_model,
    flatten_layer_resnet,
    dense_layer_1,
    dense_layer_2
])


In [10]:
resnet50_sequential_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_sequential_model.fit(trainImageResnet,validation_data=validImageResnet, epochs=10)

Epoch 1/10
256/256 [==============================] - 188s 731ms/step - loss: 4.0061 - accuracy: 0.0252 - val_loss: 4.0103 - val_accuracy: 0.0250
Epoch 2/10
256/256 [==============================] - 185s 723ms/step - loss: 3.9643 - accuracy: 0.0259 - val_loss: 3.9589 - val_accuracy: 0.0321
Epoch 3/10
256/256 [==============================] - 187s 731ms/step - loss: 3.9307 - accuracy: 0.0325 - val_loss: 3.9274 - val_accuracy: 0.0250
Epoch 4/10
256/256 [==============================] - 183s 715ms/step - loss: 3.8931 - accuracy: 0.0343 - val_loss: 3.9394 - val_accuracy: 0.0429
Epoch 5/10
256/256 [==============================] - 177s 692ms/step - loss: 3.8701 - accuracy: 0.0388 - val_loss: 3.8987 - val_accuracy: 0.0321
Epoch 6/10
256/256 [==============================] - 177s 692ms/step - loss: 3.8291 - accuracy: 0.0422 - val_loss: 3.8734 - val_accuracy: 0.0357
Epoch 7/10
256/256 [==============================] - 178s 696ms/step - loss: 3.7957 - accuracy: 0.0435 - val_loss: 3.8055 -